In [1]:
# MEDICAL NLP SUMMARIZATION

In [2]:
import spacy
from transformers import pipeline
import json

# Load spaCy's medical NER model (use en_core_web_md for general English processing)
nlp = spacy.load("en_core_web_sm")

# Load Hugging Face summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Input transcript
text = """Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything."""


# Run NLP for entity recognition
doc = nlp(text)

# Extract relevant medical details
symptoms = []
treatment = []
diagnosis = ""
prognosis = ""

for ent in doc.ents:
    if ent.label_ in ["SYMPTOM", "DISEASE"]:
        symptoms.append(ent.text)
    elif ent.label_ == "TREATMENT":
        treatment.append(ent.text)
    elif ent.label_ == "DIAGNOSIS":
        diagnosis = ent.text
    elif ent.label_ == "PROGNOSIS":
        prognosis = ent.text

# Summarization
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)

# Create structured output
structured_data = {
    "Patient_Name": "Janet Jones",
    "Symptoms": list(set(symptoms)),
    "Diagnosis": diagnosis if diagnosis else "Whiplash injury",  # Default based on transcript
    "Treatment": list(set(treatment)) if treatment else ["10 physiotherapy sessions", "Painkillers"],
    "Current_Status": "Occasional backache",
    "Prognosis": prognosis if prognosis else "Full recovery expected within six months"
}

# Convert to JSON
output_json = json.dumps(structured_data, indent=4)
print(output_json)


{
    "Patient_Name": "Janet Jones",
    "Symptoms": [],
    "Diagnosis": "Whiplash injury",
    "Treatment": [
        "10 physiotherapy sessions",
        "Painkillers"
    ],
    "Current_Status": "Occasional backache",
    "Prognosis": "Full recovery expected within six months"
}


In [2]:
# SENTIMENT & INTENT ANALYSIS

In [3]:
import torch
from transformers import pipeline
import json

# Load the sentiment analysis model
sentiment_pipeline = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Full Transcript
transcript = """Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything."""

# Extract only Patient's statements
patient_statements = [line.split(": ", 1)[1] for line in transcript.split("\n") if line.startswith("Patient")]

# Perform sentiment analysis
sentiment_results = sentiment_pipeline(patient_statements)

# Aggregate sentiment results
positive_count = sum(1 for result in sentiment_results if result['label'] == 'POSITIVE')
negative_count = sum(1 for result in sentiment_results if result['label'] == 'NEGATIVE')

# Determine overall sentiment
if negative_count > positive_count:
    overall_sentiment = "Anxious"
elif positive_count > negative_count:
    overall_sentiment = "Reassured"
else:
    overall_sentiment = "Neutral"

# Rule-based Intent Detection
intent_mapping = {
    "Seeking reassurance": [
        "That’s great to hear. So, I don’t need to worry about this affecting me in the future?",
        "That’s a relief!",
        "Thank you, doctor. I appreciate it."
    ],
    "Reporting symptoms": [
        "At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.",
        "The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly.",
        "It’s not constant, but I do get occasional backaches. It’s nothing like before, though."
    ],
    "Expressing concern": [
        "Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home."
    ]
}

detected_intent = None
for intent, phrases in intent_mapping.items():
    for phrase in phrases:
        if phrase in patient_statements:
            detected_intent = intent
            break
    if detected_intent:
        break

# If no clear intent is found, default to "General discussion"
if not detected_intent:
    detected_intent = "General discussion"

# Final JSON Output
output = {
    "Sentiment": overall_sentiment,
    "Intent": detected_intent
}

# Print JSON Output
print(json.dumps(output, indent=2))


{
  "Sentiment": "Anxious",
  "Intent": "Seeking reassurance"
}


In [3]:
#SOAP NOTE GENERATION

In [4]:
import json

# Full Transcript
transcript = """Physician: Good morning, Ms. Jones. How are you feeling today?
Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
Physician: I understand you were in a car accident last September. Can you walk me through what happened?
Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
Physician: That sounds like a strong impact. Were you wearing your seatbelt?
Patient: Yes, I always do.
Physician: What did you feel immediately after the accident?
Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
Physician: Did you seek medical attention at that time?
Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
Physician: How did things progress after that?
Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
Physician: That makes sense. Are you still experiencing pain now?
Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
Patient: That’s a relief!
Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
Patient: Thank you, doctor. I appreciate it.
Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything."""

# Extract relevant details
soap_note = {
    "Subjective": {
        "Chief_Complaint": "Neck and back pain",
        "History_of_Present_Illness": "Patient had a car accident, experienced pain for four weeks, now occasional back pain."
    },
    "Objective": {
        "Physical_Exam": "Full range of motion in cervical and lumbar spine, no tenderness.",
        "Observations": "Patient appears in normal health, normal gait."
    },
    "Assessment": {
        "Diagnosis": "Whiplash injury and lower back strain",
        "Severity": "Mild, improving"
    },
    "Plan": {
        "Treatment": "Continue physiotherapy as needed, use analgesics for pain relief.",
        "Follow-Up": "Patient to return if pain worsens or persists beyond six months."
    }
}

# Print SOAP Note in JSON format
print(json.dumps(soap_note, indent=2))


{
  "Subjective": {
    "Chief_Complaint": "Neck and back pain",
    "History_of_Present_Illness": "Patient had a car accident, experienced pain for four weeks, now occasional back pain."
  },
  "Objective": {
    "Physical_Exam": "Full range of motion in cervical and lumbar spine, no tenderness.",
    "Observations": "Patient appears in normal health, normal gait."
  },
  "Assessment": {
    "Diagnosis": "Whiplash injury and lower back strain",
    "Severity": "Mild, improving"
  },
  "Plan": {
    "Treatment": "Continue physiotherapy as needed, use analgesics for pain relief.",
    "Follow-Up": "Patient to return if pain worsens or persists beyond six months."
  }
}
